<a href="https://colab.research.google.com/github/KamonohashiPerry/MachineLearning/blob/master/BLSTM_Apartment_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTMでデザイナーズマンションの分類

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 144542 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.18-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.18-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.18-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [2]:
# drive mean root directory of  google drive
!mkdir -p drive
!google-drive-ocamlfuse drive
!ls drive/"Colab Notebooks"/NLP

alice_in_wonderland.txt  logs  RNN_Text_Generator.ipynb  si650winter11


In [1]:
# from keras.callbacks import TensorBoard
from keras.layers import Activation, Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.models import Sequential
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import collections
# import nltk
import numpy as np
# import os
# import codecs

Using TensorFlow backend.


In [0]:
import pandas as pd

In [0]:
raw_data = pd.read_csv("https://github.com/KamonohashiPerry/kamonohashiperry.com/blob/master/designers_apartment/designers_apartment.csv?raw=true")
raw_data["text"] = raw_data["text"].str.replace("デザイナーズ", "") 


In [0]:
LOG_DIR = "drive/Colab Notebooks/NLP/logs"

## 形態素解析のための準備

In [0]:
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab

In [9]:
%%bash
mecab 
俺のイタリアンにいつ行くか、今でしょう

俺	名詞,代名詞,一般,*,*,*,俺,オレ,オレ
の	助詞,連体化,*,*,*,*,の,ノ,ノ
イタリアン	名詞,一般,*,*,*,*,イタリアン,イタリアン,イタリアン
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
いつ	名詞,代名詞,一般,*,*,*,いつ,イツ,イツ
行く	動詞,自立,*,*,五段・カ行促音便,基本形,行く,イク,イク
か	助詞,副助詞／並立助詞／終助詞,*,*,*,*,か,カ,カ
、	記号,読点,*,*,*,*,、,、,、
今	名詞,副詞可能,*,*,*,*,今,イマ,イマ
でしょ	助動詞,*,*,*,特殊・デス,未然形,です,デショ,デショ
う	助動詞,*,*,*,不変化型,基本形,う,ウ,ウ
EOS


In [5]:
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.
[install-mecab-ipadic-NEologd] : Start..
[install-mecab-ipadic-NEologd] : Check the existance of libraries
[install-mecab-ipadic-NEologd] :     find => ok
[install-mecab-ipadic-NEologd] :     sort => ok
[install-mecab-ipadic-NEologd] :     head => ok
[install-mecab-ipadic-NEologd] :     cut => ok
[install-mecab-ipadic-NEologd] :     egrep => ok
[install-mecab-ipadic-NEologd] :     mecab => ok
[install-mecab-ipadic-NEologd] :     mecab-config => ok
[install-mecab-ipadic-NEologd] :     make => ok
[install-mecab-ipadic-NEologd] :     curl => ok
[install-mecab-ipadic-NEologd] :     sed => ok
[install-mecab-ipadic-NEologd] :     cat => ok
[install-mecab-ipadic-NEologd] :     diff => ok
[install-mecab-ipadic-NEologd] :   

In [6]:
!cat /etc/mecabrc

;
; Configuration file of MeCab
;
; $Id: mecabrc.in,v 1.3 2006/05/29 15:36:08 taku-ku Exp $;
;
dicdir = /var/lib/mecab/dic/debian

; userdic = /home/foo/bar/user.dic

; output-format-type = wakati
; input-buffer-size = 8192

; node-format = %m\n
; bos-format = %S\n
; eos-format = EOS\n



In [0]:
!sed -e "s!/var/lib/mecab/dic/debian!/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd!g" /etc/mecabrc > /etc/mecabrc.new

In [8]:
!cat /etc/mecabrc.new

;
; Configuration file of MeCab
;
; $Id: mecabrc.in,v 1.3 2006/05/29 15:36:08 taku-ku Exp $;
;
dicdir = /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd

; userdic = /home/foo/bar/user.dic

; output-format-type = wakati
; input-buffer-size = 8192

; node-format = %m\n
; bos-format = %S\n
; eos-format = EOS\n



In [0]:
!cp /etc/mecabrc /etc/mecabrc.org

In [0]:
!cp /etc/mecabrc.new /etc/mecabrc

In [11]:
!cat /etc/mecabrc

;
; Configuration file of MeCab
;
; $Id: mecabrc.in,v 1.3 2006/05/29 15:36:08 taku-ku Exp $;
;
dicdir = /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd

; userdic = /home/foo/bar/user.dic

; output-format-type = wakati
; input-buffer-size = 8192

; node-format = %m\n
; bos-format = %S\n
; eos-format = EOS\n



In [12]:
!apt-get -q -y install swig

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  swig3.0
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 25 not upgraded.
Need to get 1,100 kB of archives.
After this operation, 5,822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1,094 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig amd64 3.0.12-1 [6,460 B]
Fetched 1,100 kB in 0s (8,323 kB/s)
Selecting previously unselected package swig3.0.
(Reading database ... 144760 files and directories currently installed.)
Preparing to unpack .../swig3.0_3.0.12-1_amd64.deb ...
Unpacking swig3.0 (3.0.12-1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_3.0.12-1_amd64.deb ...
Unpacking swig (3.0.12

In [13]:
!pip install mecab-python3

     |████████████████████████████████| 17.1MB 204kB/s 


In [0]:
def nouns_extract(line):
    import re
    import MeCab
    keyword=[]
    m = MeCab.Tagger(' -d  /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd ')

    for l in m.parse(line).splitlines():
        if l != 'EOS' and l.split('\t')[1].split(',')[0] == '名詞':
            if l != 'EOS' and re.search('^一般$|^形容動詞語幹$|^サ変接続$|^副詞可能$|^固有名詞$' , l.split('\t')[1].split(',')[1]):
                keyword.append(l.split('\t')[0])
                
    return keyword

In [0]:
train_x, test_x, train_y, test_y = train_test_split(raw_data["text"], raw_data["designer_flag"], test_size=0.3, random_state=123)

In [22]:
train_x

246     バストイレ別 バルコニー エアコン クロゼット フローリング 浴室乾燥機 オートロック シス...
1402    バストイレ別 バルコニー エアコン ガスコンロ対応 クロゼット フローリング TVインターホ...
1586    バストイレ別 バルコニー エアコン ガスコンロ対応 クロゼット フローリング TVインターホ...
375     バストイレ別 バルコニー エアコン クロゼット フローリング 浴室乾燥機 オートロック 室内...
1272    エアコン フローリング 室内洗濯置 シューズボックス 角住戸 光ファイバー 2面採光 最上階...
                              ...                        
1122    バストイレ別 バルコニー エアコン ガスコンロ対応 クロゼット フローリング TVインターホ...
1346    バストイレ別 バルコニー エアコン ガスコンロ対応 クロゼット フローリング オートロック ...
1406    バストイレ別 バルコニー エアコン ガスコンロ対応 フローリング シャワー付洗面台 オートロ...
1389    バストイレ別 エアコン クロゼット フローリング シューズボックス 押入 光ファイバー 即入...
1534    バルコニー エアコン クロゼット TVインターホン オートロック 陽当り良好 シューズボック...
Name: text, Length: 1304, dtype: object

In [23]:
train_y

246     1
1402    0
1586    0
375     1
1272    0
       ..
1122    0
1346    0
1406    0
1389    0
1534    0
Name: designer_flag, Length: 1304, dtype: int64

In [0]:
# 語彙に関するデータを作成するための処理

maxlen = 0
word_freqs = collections.Counter()
num_recs = 0

for sentence in list(train_x):
  words = nouns_extract(sentence)
  maxlen = max(maxlen, len(words)) # リストの最大値を取得
  # 単語の出現頻度を集計
  for word in words:
    word_freqs[word] += 1
  num_recs += 1

In [25]:
print(maxlen)

108


In [26]:
print(len(word_freqs))

360


In [27]:
num_recs

1304

In [0]:
# 語彙のなかから2000語を選ぶ
MAX_FEATURES = 2000

# 文の長さの最大値
MAX_SENTENCE_LENGTH = 40

# 語彙数
vocab_size = min(MAX_FEATURES, len(word_freqs)) + 2

In [18]:
vocab_size

363

In [0]:
# 単語とそのインデックスからなるdict形式のデータ
word2index = {x[0]: i+2 for i, x in enumerate(word_freqs.most_common(MAX_FEATURES))}

# 0番目と1番目にパディング用(PAD)と擬似単語(UNK)を指定する
# パディングとは系列の長さを揃えること。
word2index['PAD'] = 0
word2index['UNK'] = 1

# インデックスとその単語からなるdict形式のデータ
index2word = {v:k for k, v in word2index.items()}

In [20]:
np.empty((num_recs, ), dtype=list).shape

(1304,)

In [0]:
# 空の変数を作成
X = np.empty((num_recs, ), dtype=list)
y = np.zeros((num_recs, ))
i = 0

train_x_list = list(train_x)
train_y_list = list(train_y)

for i in range(len(train_x_list)):
  label = train_y_list[i]
  words = nouns_extract(train_x_list[i])
  # 特徴量用の空の変数
  seqs = []
  for word in words:
    # 単語を照合してインデックスを取得
    if word in word2index:
      seqs.append(word2index[word])
    else:
      # 語彙に存在しない単語の場合はUNKのインデックスを付与する
      seqs.append(word2index["UNK"])
  
  X[i] = seqs
  y[i] = int(label)
  i += 1

# パディングして、同じ長さに揃えてくれる関数
X = sequence.pad_sequences(X, maxlen=MAX_SENTENCE_LENGTH)

In [35]:
X

array([[  0,   0,   0, ...,  39,  20,  48],
       [119,   3,  32, ...,  65,  66,  67],
       [ 50,  70,  70, ...,  65,  66,  67],
       ...,
       [  0,   0,   0, ...,  23,   9,  28],
       [  0,   0,   0, ...,  65,  66,  67],
       [ 68,  29,  22, ...,  65,  66,  67]], dtype=int32)

In [36]:
vocab_size

362

In [0]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42)

In [0]:
import tensorflow as tf
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback, EarlyStopping


# define roc_callback, inspired by https://github.com/keras-team/keras/issues/6050#issuecomment-329996505
def auc_roc(y_true, y_pred):
    # any tensorflow metric
    value, update_op = tf.contrib.metrics.streaming_auc(y_pred, y_true)

    # find all variables created for this metric
    metric_vars = [i for i in tf.local_variables() if 'auc_roc' in i.name.split('/')[1]]

    # Add metric variables to GLOBAL_VARIABLES collection.
    # They will be initialized for new session.
    for v in metric_vars:
        tf.add_to_collection(tf.GraphKeys.GLOBAL_VARIABLES, v)

    # force to update metric values
    with tf.control_dependencies([update_op]):
        value = tf.identity(value)
        return value

In [0]:
# 分散表現層の次元
EMBEDDING_SIZE = 128
# 隠れ層の数
HIDDEN_LAYER_SIZE = 64
# バッチサイズ
BATCH_SIZE = 32
# エポックの数
NUM_EPOCHS = 10

model = Sequential()
# 分散表現の層
model.add(Embedding(vocab_size, EMBEDDING_SIZE, input_length=MAX_SENTENCE_LENGTH))
# Bidirectional LSTMの層
model.add(Bidirectional(LSTM(HIDDEN_LAYER_SIZE )))
# ドロップアウト層
model.add(Dropout(0.5))
# 活性化層
model.add(Dense(1, activation='sigmoid'))
# モデルのコンパイル
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy", auc_roc])

In [40]:
my_callbacks = [EarlyStopping(monitor='auc_roc', patience=300, verbose=1, mode='max')]

# ネットワークの学習
history = model.fit(Xtrain, ytrain, batch_size=BATCH_SIZE,
                    epochs=NUM_EPOCHS,
                    callbacks=my_callbacks,
                    validation_data=(Xtest, ytest))

Train on 1043 samples, validate on 261 samples
Epoch 1/10
1043/1043 [==============================] - 8s 7ms/step - loss: 0.5156 - acc: 0.8140 - auc_roc: 0.5284 - val_loss: 0.4298 - val_acc: 0.8391 - val_auc_roc: 0.5793
Epoch 2/10
1043/1043 [==============================] - 5s 5ms/step - loss: 0.3924 - acc: 0.8293 - auc_roc: 0.6271 - val_loss: 0.4221 - val_acc: 0.8506 - val_auc_roc: 0.6631
Epoch 3/10
1043/1043 [==============================] - 5s 5ms/step - loss: 0.3646 - acc: 0.8514 - auc_roc: 0.6898 - val_loss: 0.4020 - val_acc: 0.8467 - val_auc_roc: 0.7089
Epoch 4/10
1043/1043 [==============================] - 6s 6ms/step - loss: 0.3275 - acc: 0.8696 - auc_roc: 0.7296 - val_loss: 0.4086 - val_acc: 0.8314 - val_auc_roc: 0.7453
Epoch 5/10
1043/1043 [==============================] - 6s 5ms/step - loss: 0.2959 - acc: 0.8878 - auc_roc: 0.7614 - val_loss: 0.3821 - val_acc: 0.8582 - val_auc_roc: 0.7736
Epoch 6/10
1043/1043 [==============================] - 6s 5ms/step - loss: 0.2779 

In [0]:
# %load_ext tensorboard

In [0]:
# %tensorboard --logdir='drive/Colab Notebooks/NLP/logs'

In [41]:
# testデータでの予測
score, acc, auc = model.evaluate(Xtest, ytest, batch_size=BATCH_SIZE)
print("Test score: {:.3f}, accuracy: {:.3f}, auc: {:.3f}".format(score, acc, auc))

for i in range(20):
  idx = np.random.randint(len(Xtest))
  xtest = Xtest[idx].reshape(1, 40)
  ylabel = ytest[idx]
  ypred = model.predict(xtest)[0][0]
  sent = " ".join([index2word[x] for x in xtest[0].tolist() if x != 0 ])
  print("{:.0f}\t{:.0f}\t{}".format(ypred, ylabel, sent))

261/261 [==============================] - 0s 2ms/step
Test score: 0.444, accuracy: 0.824, auc: 0.844
0	0	バルコニー エアコン クロゼット フローリング 室内 洗濯 置 シューズボックス システムキッチン 住戸 外壁 タイル 入居 ガスレンジ 都市ガス
0	0	ウォークインクロゼット 保証 不要 敷金 バイク 置場 ネット 専用 回線 トランクルーム 事務所 相談 楽器 相談 トイレ ルームシェア 相談 利用 沿線 利用 徒歩 10分 専有 面積 平面 駐車場 LDK 居室 都市ガス 高速 ネット 対応 礼金 保証 会社 利用 初期 費用 カード 決済
0	0	バス トイレ エアコン ガスコンロ 対応 クロゼット フローリング 室内 洗濯 置 シューズボックス システムキッチン 脱衣所 入居 礼金 不要 最上階 敷金 ガスレンジ ダブル ロック キー 上 都市ガス
0	0	室内 洗濯 置 焚 機能 浴室 住戸 脱衣所 洗面所 独立 駐輪場 光ファイバー 入居 ペット 相談 敷金 居室 フローリング 採光 収納 内装 リフォーム 済 事務所 相談 ルームシェア 相談 利用 沿線 利用 徒歩 5分 東南 都市ガス 玄関 収納 礼金 保証 会社 利用
0	0	居室 フローリング 採光 沿線 利用 内装 リフォーム 済 眺望 良好 事務所 相談 24時間 換気 システム 南面 リビング 外装 リフォーム 済 利用 利用 沿線 利用 徒歩 10分 敷地内 ごみ 置き場 都市ガス BS 保証 会社 利用 初期 費用 カード 決済 通風 良好
0	0	TV インターホン 浴室 乾燥機 オートロック 室内 洗濯 置 システムキッチン 住戸 エレベーター 洗面所 独立 コンロ 宅配ボックス 光ファイバー 外壁 タイル BS CS 防犯カメラ 保証 不要 敷金 保証金 不要 プール 利用 沿線 利用 徒歩 10分 24時間 ゴミ 敷地内 ごみ 置き場 5年 都市ガス 礼金
0	0	バルコニー エアコン クロゼット フローリング 室内 洗濯 置 シューズボックス システムキッチン 住戸 外壁 タイル 入居 ガスレンジ 都市ガス
0	1	バス トイレ バルコニー エアコン クロゼット 

## LSTMバージョン

In [42]:
# 分散表現層の次元
EMBEDDING_SIZE = 128
# 隠れ層の数
HIDDEN_LAYER_SIZE = 64
# バッチサイズ
BATCH_SIZE = 32
# エポックの数
NUM_EPOCHS = 10

model = Sequential()
# 分散表現の層
model.add(Embedding(vocab_size, EMBEDDING_SIZE, input_length=MAX_SENTENCE_LENGTH))
# LSTMの層
model.add(LSTM(HIDDEN_LAYER_SIZE ))
# ドロップアウト層
model.add(Dropout(0.5))
# 活性化層
model.add(Dense(1, activation='sigmoid'))
# モデルのコンパイル
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy", auc_roc])

my_callbacks = [EarlyStopping(monitor='auc_roc', patience=300, verbose=1, mode='max')]

# ネットワークの学習
history = model.fit(Xtrain, ytrain, batch_size=BATCH_SIZE,
                    epochs=NUM_EPOCHS,
                    callbacks=my_callbacks,
                    validation_data=(Xtest, ytest))

Train on 1043 samples, validate on 261 samples
Epoch 1/10
1043/1043 [==============================] - 4s 4ms/step - loss: 0.5176 - acc: 0.8054 - auc_roc: 0.5189 - val_loss: 0.4234 - val_acc: 0.8391 - val_auc_roc: 0.5769
Epoch 2/10
1043/1043 [==============================] - 3s 3ms/step - loss: 0.4104 - acc: 0.8274 - auc_roc: 0.6131 - val_loss: 0.4003 - val_acc: 0.8391 - val_auc_roc: 0.6456
Epoch 3/10
1043/1043 [==============================] - 2s 2ms/step - loss: 0.3661 - acc: 0.8476 - auc_roc: 0.6813 - val_loss: 0.3930 - val_acc: 0.8314 - val_auc_roc: 0.7060
Epoch 4/10
1043/1043 [==============================] - 3s 3ms/step - loss: 0.3255 - acc: 0.8600 - auc_roc: 0.7264 - val_loss: 0.3928 - val_acc: 0.8467 - val_auc_roc: 0.7457
Epoch 5/10
1043/1043 [==============================] - 2s 2ms/step - loss: 0.2985 - acc: 0.8830 - auc_roc: 0.7620 - val_loss: 0.3971 - val_acc: 0.8352 - val_auc_roc: 0.7745
Epoch 6/10
1043/1043 [==============================] - 3s 3ms/step - loss: 0.2780 

In [43]:
# testデータでの予測
score, acc, auc = model.evaluate(Xtest, ytest, batch_size=BATCH_SIZE)
print("Test score: {:.3f}, accuracy: {:.3f}, auc: {:.3f}".format(score, acc, auc))

for i in range(20):
  idx = np.random.randint(len(Xtest))
  xtest = Xtest[idx].reshape(1, 40)
  ylabel = ytest[idx]
  ypred = model.predict(xtest)[0][0]
  sent = " ".join([index2word[x] for x in xtest[0].tolist() if x != 0 ])
  print("{:.0f}\t{:.0f}\t{}".format(ypred, ylabel, sent))

261/261 [==============================] - 0s 1ms/step
Test score: 0.445, accuracy: 0.839, auc: 0.841
0	1	洗濯機 エアコン 室 24時間 換気 システム 電子レンジ 学生 相談 ベッド 家電 家具付 耐火構造 耐震 構造 カーテン 利用 沿線 利用 徒歩 10分 高層 24時間 ゴミ 敷地内 ごみ 置き場 都市ガス 洗面所 ドア 玄関 収納 高速 ネット 対応 礼金 テレビ ダイニング テーブル デスク
0	0	シャワー 洗面 TV インターホン 浴室 乾燥機 室内 洗濯 置 シューズボックス システムキッチン 住戸 脱衣所 洗面所 独立 採光 IH クッキング ヒーター 照明 保証人 不要 敷金 沿線 利用 住戸 利用 利用 沿線 利用 徒歩 5分 駅 徒歩 10分 東南 都市ガス 洗面所 ドア 礼金
1	0	防犯カメラ ペット 相談 IH クッキング ヒーター 仲介手数料 不要 敷金 二人 入居 相談 居室 フローリング CS 保証 不要 浴室 窓 事務所 相談 照明 センサー 利用 沿線 利用 徒歩 5分 駅 徒歩 10分 24時間 ゴミ 敷地内 ごみ 置き場 BS 保証 会社 利用
0	0	タイル 入居 敷金 不要 防犯カメラ 分譲 賃貸 沿線 利用 ネット 専用 回線 平坦 保証 不要 利用 徒歩 5分 駅 徒歩 10分 タワー マンション 24時間 ゴミ 敷地内 ごみ 置き場 都市ガス BS 高速 ネット 対応 年内 入居 礼金 初期 費用 カード 決済
0	0	オートロック 室内 洗濯 置 シューズボックス システムキッチン 南 駐輪場 CATV 光ファイバー 閑静 住宅地 敷金 24時間 緊急通報 システム 沿線 利用 クッション フロア 平坦 床下 収納 保証 不要 利用 徒歩 10分 敷地内 ごみ 置き場 当社 管理 物件 都市ガス 年度内 入居 礼金 全 照明
0	0	換気 システム 高温 湯 楽器 相談 LDK エアコン トイレ クロゼット 南面 リビング バルコニー 両面 バルコニー 玄関 ポーチ エアコン 利用 利用 沿線 利用 徒歩 10分 平面 駐車場 2階 LDK 居室 都市ガス 南面 バルコニー 礼金 保証 会社 利用 初期 